# &#x1F4DD; REPORT

[CompMeth](https://www.eurecom.fr/en/course/compmeth-2024spring) Spring 2024

Computational methods in digital communications provide a selection of hands-on experiments in programming and implementation techniques for high-performance computing applied to telecommunications. Students will learn architecture concepts and how to optimize software to efficiently implement different types of algorithms on software-based systems.

Pedagogical Outcomes: Students will gain knowledge of some generic digital programming methods and a deeper understanding of a topic in the context of a mini-project. They will also learn to work in teams and to synthesize their understanding through a presentation at the end of the course in front of all students.

---

#### **&#x1F516;** **Project** ___The Open Air Interface___

Project Location: [openairinterface5g](https://gitlab.eurecom.fr/oai/openairinterface5g)

Sub Project: [DMRS Channel Estimation Parallelization](https://gitlab.eurecom.fr/oai/openairinterface5g/-/tree/dmrs_channel_estimation_parallelization/openair1/PHY/NR_ESTIMATION)

Description: Using the current OAI's custom [Thread Pool](https://gitlab.eurecom.fr/oai/openairinterface5g/-/blob/develop/common/utils/threadPool/thread-pool.md), parralelize the existing NR (New Radio) UL (UpLink) Channel Estimation.

---

In the context of an uplink channel in a wireless communication system such as LTE (Long Term Evolution) or 5G NR (New Radio), DMRS (Demodulation Reference Signal) channel estimation is used to estimate the characteristics of the uplink channel. DMRS channel estimation in the uplink is a vital process in wireless communication systems, enabling the base station to accurately estimate the uplink channel conditions. This estimation allows for coherent demodulation, adaptive resource allocation, and advanced MIMO techniques, ensuring efficient and reliable communication.



#### **&#x1F516;** **(&#x61;)** ___Coding Practice___

---

Caution has been used not to break the existing `functionning` code. The coding effort relies on 4 steps:

### **(&#x31;)** Separating the main channel estimation code per antenna

- `function nr_pusch_channel_estimation()` [&#x1F4DC; Source Code](https://gitlab.eurecom.fr/oai/openairinterface5g/-/blob/dmrs_channel_estimation_parallelization/openair1/PHY/NR_ESTIMATION/nr_ul_channel_estimation.c?ref_type=heads#L521)

```c
int nr_pusch_channel_estimation(PHY_VARS_gNB *gNB, ...)
{

    //------------------generate DMRS------------------//
    <some code here>
    
    for (int aarx=0; aarx<gNB->frame_parms.nb_antennas_rx; aarx++) {
    
        // refactor the channel estimation code to a function
        
    }

 return 0;
}

```

- new function `inner_channel_estimation` {[&#x1F4DC; Source Code](https://gitlab.eurecom.fr/oai/openairinterface5g/-/blob/dmrs_channel_estimation_parallelization/openair1/PHY/NR_ESTIMATION/nr_ul_channel_estimation.c?ref_type=heads#L70)

```c
static void inner_channel_estimation(void *arg) {
    ...
}
```

&#x1F4A1; The code is still sequential

### **(&#x32;)** Implement the Thread Pool using the new `inner_channel_estimation` function


- Initialize `local` data to be passed to the threadpool,

```c
 int nb_antennas_rx = gNB->frame_parms.nb_antennas_rx;
 delay_t *delays[nb_antennas_rx];
 memset(delays, 0, sizeof(*delays));

 uint64_t noises_amp2[nb_antennas_rx];
 memset(noises_amp2, 0, sizeof(noises_amp2));
```

- In the antenna loop,
    - call the TPool, by passing the function to the worker
    - pass data `(*dtata)` to the worker
    - wait for the TPool join to complete

```c
for (int aarx=0; aarx<gNB->frame_parms.nb_antennas_rx; aarx++) {

   notifiedFIFO_elt_t *req = newNotifiedFIFO_elt(sizeof(puschAntennaProc_t), aarx, &gNB->respPuschAarx, &inner_channel_estimation); // create a job for Tpool
   puschAntennaProc_t *rdata = (puschAntennaProc_t*)NotifiedFifoData(req); // data for the job

   // Local init in the current loop
   rdata->aarx = aarx;
   rdata->noises_amp2 = noises_amp2;   // Placeholder for noise amplitude squared update
   rdata->delays = delays;

   ...

   // Call the inner_channel_estimation function
   //   inner_channel_estimation(rdata);
   pushTpool(&gNB->threadPool, req);
   gNB->nbAarx++;


   // value update of rdata to be passed to the next inner call
   delay->est_delay = rdata->delay->est_delay;  // Placeholder for estimated delay update
   ...

} // Antenna Loop

 // ThreadPool Join (also called Rendez-Vous)
 while (gNB->nbAarx > 0) {
   notifiedFIFO_elt_t *req = pullTpool(&gNB->respPuschAarx, &gNB->threadPool);
   gNB->nbAarx--;
   delNotifiedFIFO_elt(req);
 }
```

### **(&#x32;)** Adding traces to the code

To make sure the data is passed (bach and forth) through the `Thread Pool` a trace has been added and will be removed.

- [ ] Define the trace

```
#include <inttypes.h>
#define DEBUG_PUSCH_THREAD
```

- [ ] Implement tracing

* Trace after initialization

```c
#ifdef DEBUG_PUSCH_THREAD

 for (int aarx=0; aarx<gNB->frame_parms.nb_antennas_rx; aarx++) {
    printf("Start - Estimated delay = %i\t", delay[aarx].est_delay >> 1);
 }

#endif

* Trace at the Thread Pool exit


```c
 #ifdef DEBUG_PUSCH_THREAD

 printf("\n Exit Pool - Starts with: %i\n",gNB->frame_parms.nb_antennas_rx);
 for (int aarx=0; aarx<gNB->frame_parms.nb_antennas_rx; aarx++) {
  printf("Array # = %i\t Estimated delay = %i\t Noise Amp2 = %" PRIu64 "\t", aarx, delay[aarx].est_delay, noises_amp2[aarx]);
 }
 printf("\n Exit Pool - Ends \n");

#endif
```


#### **&#x1F516;** **(&#x62;)** ___Compiling and Running the tests___

---



#### **(&#x33;)** Build the code using the PHY Simulators

- [ ] Build

```
./cmake_targets/build_oai --phy_simulators
```

- [ ] Test

* When testing with a single antenna (-z 1)

```
sudo cmake_targets/ran_build/build/nr_ulsim -C 4 -m 25 -s 24 -z 1 -n 100 -P -q 1 -R 273 -r 273
```

* When testing with a multiple antennas (-z 4)

```
sudo cmake_targets/ran_build/build/nr_ulsim -C 4 -m 25 -s 24 -z 4 -n 100 -P -q 1 -R 273 -r 273
```

#### **&#x1F516;** **(&#x63;)** ___Impacted source code___

---


- [ ] Headers

[defs_gNB.h](https://gitlab.eurecom.fr/oai/openairinterface5g/-/blob/dmrs_channel_estimation_parallelization/openair1/PHY/defs_gNB.h)

```c
typedef struct {
  PHY_VARS_gNB *gNB;
  int aarx;

...

} puschAntennaProc_t;
```

```c
/// Top-level PHY Data Structure for gNB
typedef struct PHY_VARS_gNB_s {
  /// Module ID indicator for this instance
  module_id_t          Mod_id;

...
  notifiedFIFO_t respPuschAarx;
...
  int nbAarx;
...
} PHY_VARS_gNB;
```

- [ ] Source Code

[nr_ul_channel_estimation.c](https://gitlab.eurecom.fr/oai/openairinterface5g/-/blob/dmrs_channel_estimation_parallelization/openair1/PHY/NR_ESTIMATION/nr_ul_channel_estimation.c)

See **&#x1F516;** **(&#x61;)** for coding implementation

#### **&#x1F516;** **(&#x64;)** ___Debugging___

---

#### **&#x1F516;** **(&#x65;)** ___Results___

---